In [132]:
import boto3
#import matplotlib.pyplot as plt
#%matplotlib inline
import IPython
from IPython.display import JSON
import json
from IPython.display import display
import IPython.display as ipd

In [133]:

textSample = """
    
    Nordic and Baltic stock markets were halted by technical problems for a second time on Friday, only minutes after trading resumed following earlier problems.

    The equity and equity derivatives markets had reopened at 12:00 GMT after a two-hour trading halt that Nasdaq attributed to connectivity issues.

    “Due to technical disturbances, Nasdaq Nordic Equity and Nasdaq Nordic Index and Equity Derivatives markets (have been) halted again,” operator Nasdaq said in an emailed statement.

    The company operates bourses in Finland, Denmark, Sweden, Iceland, Estonia, Latvia and Lithuania.

    Nasdaq said after the second stoppage that the Stockholm bourse, which had already been scheduled to close early ahead of a public holiday, and the Baltic exchanges would remain closed for the rest of the day.

    Trading in Copenhagen, Helsinki and Reykjavik will resume with opening auctions at 15:05 GMT, it said, followed by continuous trading from 1615 CET (1515 GMT).

    Those bourses are scheduled to close as per normal trading hours.

    “We are working diligently to correct the technical issue we experienced with our Nordic and Baltic markets today,” Lauri Rosendahl, head of European equities and derivatives at Nasdaq, said in an email.

    “We apologize for the inconvenience this has created for our exchange members and all investors in our markets.”

    Trading on the Norwegian bourse, which is owned by Euronext, was not interrupted.
    

    """


In [134]:
def synthesizeText(text, engine="neural", languageCode="en-US", VoiceId = "Joanna"):
    client = boto3.client('polly')
    response = client.synthesize_speech(
        Engine=engine,
        LanguageCode=languageCode,
        OutputFormat='mp3',
        #SpeechMarkTypes=['ssml', 'sentence', 'viseme', 'word'],
        Text=text,
        VoiceId=VoiceId
    )
    return response


In [135]:
pollyresults = synthesizeText(textSample)


In [136]:
print(pollyresults)

{'ResponseMetadata': {'RequestId': '177e2ca9-086b-4b0e-819a-1249a7f330d2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '177e2ca9-086b-4b0e-819a-1249a7f330d2', 'x-amzn-requestcharacters': '1416', 'content-type': 'audio/mpeg', 'transfer-encoding': 'chunked', 'date': 'Sun, 03 Nov 2019 12:34:19 GMT'}, 'RetryAttempts': 0}, 'ContentType': 'audio/mpeg', 'RequestCharacters': '1416', 'AudioStream': <botocore.response.StreamingBody object at 0x7f4b95c26438>}


In [137]:
file = open('pollyspeech.mp3', 'wb')
file.write(pollyresults['AudioStream'].read())
file.close()

In [138]:
#!ls
ipd.Audio('pollyspeech.mp3')

In [139]:
def translate(text, sourcelang="en", targetlang="fr"):
    client = boto3.client('translate')
    response = client.translate_text(
        Text=text,
        SourceLanguageCode=sourcelang,
        TargetLanguageCode=targetlang
    )
    return response

In [140]:
translated = translate(textSample)
print(json.dumps(translated, indent=2))

{
  "TranslatedText": "Les march\u00e9s boursiers nordiques et baltes ont \u00e9t\u00e9 interrompus par des probl\u00e8mes techniques pour une deuxi\u00e8me fois vendredi, quelques minutes seulement apr\u00e8s la reprise de la n\u00e9gociation suite \u00e0 des probl\u00e8mes ant\u00e9rieurs.\n\n Les march\u00e9s des actions et des d\u00e9riv\u00e9s d'actions avaient rouvert \u00e0 12 heures GMT apr\u00e8s un arr\u00eat de n\u00e9gociation de deux heures attribu\u00e9 par le Nasdaq \u00e0 des probl\u00e8mes de connectivit\u00e9.\n\n \u00ab\u00a0En raison de perturbations techniques, les march\u00e9s Nasdaq Nordic Equity et Nasdaq Nordic Index and Equity Derivatives (ont \u00e9t\u00e9) arr\u00eat\u00e9s \u00e0 nouveau\u00a0\u00bb, a d\u00e9clar\u00e9 l'op\u00e9rateur Nasdaq dans un communiqu\u00e9 envoy\u00e9 par e-mail.\n\n La soci\u00e9t\u00e9 exploite des bourses en Finlande, au Danemark, en Su\u00e8de, en Islande, en Estonie, en Lettonie et en Lituanie.\n\n Nasdaq a d\u00e9clar\u00e9

In [141]:
pollyresults = synthesizeText(translated["TranslatedText"], engine="standard", languageCode="fr-CA", VoiceId = "Chantal")

In [142]:
print(pollyresults)

{'ResponseMetadata': {'RequestId': '6fd5d465-0c6e-409c-b661-af4b96846a5b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '6fd5d465-0c6e-409c-b661-af4b96846a5b', 'x-amzn-requestcharacters': '1681', 'content-type': 'audio/mpeg', 'transfer-encoding': 'chunked', 'date': 'Sun, 03 Nov 2019 12:34:23 GMT'}, 'RetryAttempts': 0}, 'ContentType': 'audio/mpeg', 'RequestCharacters': '1681', 'AudioStream': <botocore.response.StreamingBody object at 0x7f4b95bb7908>}


In [143]:
file = open('pollyspeechv2.mp3', 'wb')
file.write(pollyresults['AudioStream'].read())
file.close()

In [144]:
ipd.Audio('pollyspeechv2.mp3')

In [145]:

def detect_dominant_language(text):
    client = boto3.client('comprehend')
    response = client.detect_dominant_language(
        Text=text
    )    
    language = response["Languages"][0]["LanguageCode"]
    print("language: ", language )
    #print("languages: ", response)
    return language

def detect_entities(text):
    client = boto3.client('comprehend')
    language = detect_dominant_language(text)
    response = client.detect_entities(Text='string', LanguageCode=language)
    print(json.dumps(response, indent=2))
    return response
    #print(response)

def detect_key_phrases(text):
    client = boto3.client('comprehend') 
    language = detect_dominant_language(text)
    response = client.detect_key_phrases(
            Text=text,
            LanguageCode=language
        )    
    print(json.dumps(response, indent=2))
    return response    
    
def detect_sentiment(text):
    client = boto3.client('comprehend') 
    language = detect_dominant_language(text)
    response = client.detect_sentiment(
            Text=text,
            LanguageCode=language
        )    
    print(json.dumps(response, indent=2))
    return response      

def detect_syntax(text):
    client = boto3.client('comprehend') 
    language = detect_dominant_language(text)
    response = client.detect_syntax(
            Text=text,
            LanguageCode=language
        )    
    print(json.dumps(response, indent=2))
    return response      

In [146]:
entities = detect_entities(textSample)

language:  en
{
  "Entities": [],
  "ResponseMetadata": {
    "RequestId": "adf58351-54f9-4f0d-9247-9b3e8073e4e2",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "adf58351-54f9-4f0d-9247-9b3e8073e4e2",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "15",
      "date": "Sun, 03 Nov 2019 12:34:23 GMT"
    },
    "RetryAttempts": 0
  }
}


In [147]:
keyphrases = detect_key_phrases(textSample)

language:  en
{
  "KeyPhrases": [
    {
      "Score": 0.9347984194755554,
      "Text": "Nordic and Baltic stock markets",
      "BeginOffset": 10,
      "EndOffset": 41
    },
    {
      "Score": 0.9997931718826294,
      "Text": "technical problems",
      "BeginOffset": 57,
      "EndOffset": 75
    },
    {
      "Score": 0.9994344115257263,
      "Text": "a second time",
      "BeginOffset": 80,
      "EndOffset": 93
    },
    {
      "Score": 0.9639489650726318,
      "Text": "Friday",
      "BeginOffset": 97,
      "EndOffset": 103
    },
    {
      "Score": 0.9687023758888245,
      "Text": "only minutes",
      "BeginOffset": 105,
      "EndOffset": 117
    },
    {
      "Score": 0.7657560110092163,
      "Text": "trading",
      "BeginOffset": 124,
      "EndOffset": 131
    },
    {
      "Score": 0.9982903003692627,
      "Text": "earlier problems",
      "BeginOffset": 150,
      "EndOffset": 166
    },
    {
      "Score": 0.9831377863883972,
      "Text": "The equit

In [148]:
sentiment = detect_sentiment(textSample)

language:  en
{
  "Sentiment": "NEGATIVE",
  "SentimentScore": {
    "Positive": 0.004468288738280535,
    "Negative": 0.7433950304985046,
    "Neutral": 0.2520737946033478,
    "Mixed": 6.290546298259869e-05
  },
  "ResponseMetadata": {
    "RequestId": "3b41245b-d5d1-43fd-ba72-20333d54491a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "3b41245b-d5d1-43fd-ba72-20333d54491a",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "163",
      "date": "Sun, 03 Nov 2019 12:34:24 GMT"
    },
    "RetryAttempts": 0
  }
}


In [149]:
syntax = detect_syntax(textSample)

language:  en
{
  "SyntaxTokens": [
    {
      "TokenId": 1,
      "Text": "Nordic",
      "BeginOffset": 10,
      "EndOffset": 16,
      "PartOfSpeech": {
        "Tag": "ADJ",
        "Score": 0.8861039876937866
      }
    },
    {
      "TokenId": 2,
      "Text": "and",
      "BeginOffset": 17,
      "EndOffset": 20,
      "PartOfSpeech": {
        "Tag": "CONJ",
        "Score": 0.9998586177825928
      }
    },
    {
      "TokenId": 3,
      "Text": "Baltic",
      "BeginOffset": 21,
      "EndOffset": 27,
      "PartOfSpeech": {
        "Tag": "ADJ",
        "Score": 0.8909648060798645
      }
    },
    {
      "TokenId": 4,
      "Text": "stock",
      "BeginOffset": 28,
      "EndOffset": 33,
      "PartOfSpeech": {
        "Tag": "NOUN",
        "Score": 0.9993277788162231
      }
    },
    {
      "TokenId": 5,
      "Text": "markets",
      "BeginOffset": 34,
      "EndOffset": 41,
      "PartOfSpeech": {
        "Tag": "NOUN",
        "Score": 0.9999920129776001
    